  # DataLoader and Dataset

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [12]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [13]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X,y in test_dataloader:
    print(f"Shape of X[N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [21]:
for X,y in test_dataloader:
    print(X)
    print(y)
    print(X.grad)
    print(y.grad)
    break

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

# Creating Models

In [15]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_avaliable()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


WK: 
nn.Flatten(): 扁平化处理
nn.Sequential(): 模型序列
nn.Linear: linear transformation $y=xA^T+b$
nn.ReLU(rectified linear unit function)激活函数 $ReLU(x)=(x)^+ = max(0,x)$


In [16]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Optimizing
loss function and optimizer

* nn.CrossEntropyLoss 交叉熵损失函数
* torch.optim.SGD (Stochastic Gradient Descent)

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

* model.train() 设置模型为训练模式
* Tensor.backward() 关于叶节点计算梯度 [解释](https://zhuanlan.zhihu.com/p/592176365)
* optimizer.step() 执行一步优化
* optimizer.zero_grad() 重置梯度

In [18]:
def train(dataLoader:DataLoader, model:nn.Module, loss_fn, optimizer):
    size = len(dataLoader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataLoader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

* model.eval() 设置模型为 evaluation 模式
* torch.no_grad() 上下文，用以停止梯度计算，用以减少内存消耗

In [19]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306227  [   64/60000]
loss: 2.300326  [ 6464/60000]
loss: 2.276404  [12864/60000]
loss: 2.267387  [19264/60000]
loss: 2.265606  [25664/60000]
loss: 2.219690  [32064/60000]
loss: 2.233914  [38464/60000]
loss: 2.199520  [44864/60000]
loss: 2.197933  [51264/60000]
loss: 2.165333  [57664/60000]
Test Error: 
 Accuracy: 29.3%, Avg loss: 0.033975 

Epoch 2
-------------------------------
loss: 2.175581  [   64/60000]
loss: 2.179806  [ 6464/60000]
loss: 2.119908  [12864/60000]
loss: 2.129442  [19264/60000]
loss: 2.102808  [25664/60000]
loss: 2.021377  [32064/60000]
loss: 2.060403  [38464/60000]
loss: 1.988017  [44864/60000]
loss: 1.991624  [51264/60000]
loss: 1.918264  [57664/60000]
Test Error: 
 Accuracy: 57.6%, Avg loss: 0.030221 

Epoch 3
-------------------------------
loss: 1.961856  [   64/60000]
loss: 1.943623  [ 6464/60000]
loss: 1.831151  [12864/60000]
loss: 1.857702  [19264/60000]
loss: 1.770739  [25664/60000]
loss: 1.700946  [32064/600